<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [1]:
from IPython.core.display import HTML
HTML(open('css/custom.css', 'r').read())

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2021 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 6.</h5>

<h1 class='lesson_title'>Solving Dynamic Programs with NetworkX</h1>

## Overview

* In this lesson, we'll revisit a few examples of dynamic programs and solve them with NetworkX.

## The knapsack problem, revisited

You are a thief deciding which precious metals to steal from a vault:
                                  
|    | Metal    | Weight (kg) | Value |
|:---|:---------|:------------|:------|
| 1  | Gold     | 3           | 11    |
| 2  | Silver   | 2           | 7     |
| 3  | Platinum | 4           | 12    |
                                  
You have a knapsack that can hold at most 8 kg. If you decide to take a particular metal, you must take all of it. Which items should you take to maximize the value of your theft?

* Recall that we formulated this problem as a dynamic program with the following longest path representation:
    - Stage $t$ represents the decision to take item $t$ ($t = 1, 2, 3$), or the end of the decision-making process ($t = 4$).
    - Node $t_n$ represents having $n$ kgs left in knapsack at stage $t$ ($n = 0, 1, \dots, 8$).

![DP for knapsack example](img/knapsack.png)

* We know how to solve shortest/longest path problems using NetworkX, so we can apply the same ideas here.

* There is a Python data structure that makes this a little easier...

### Tuples

* A __tuple__ is like a list, except once it's been defined, it cannot be changed.

* A tuple is written as a sequence of comma-separated items between _round_ brackets. For example:

In [ ]:
# Define a tuple corresponding to taking 
# silver with 5 kgs left in the knapsack


In [2]:
# Solution
# Define a tuple corresponding to taking 
# silver with 5 kgs left in the knapsack
stage = (2, 5)

* Tuples are ideal for things like names of nodes &mdash; things that you want to make permanent and  not accidentally change.

### Back to the knapsack problem...

* We can use a tuple to represent the name of each node in our dynamic program, since each node's name has two distinct parts: the stage and the state.

* Before we do anything, we need to import `networkx` and `bellmanford`:

In [3]:
import networkx as nx
import bellmanford as bf

* Let's begin by creating an empty graph:

In [ ]:
# Create empty graph


In [4]:
# Solution
# Create empty graph
G = nx.DiGraph()

* Next, let's add the stage-state nodes to the graph, using `for` loops. Remember that `range(a, b)` iterates over the integers `a, a + 1, ..., b - 1`.

In [ ]:
# Add the stage-state nodes


In [5]:
# Solution
# Add the stage-state nodes
for t in range(1, 5):
    for n in range(0, 9):
        G.add_node((t, n))

* We also need to add the special "end" node:

In [ ]:
# Add the end node


In [6]:
# Solution
# Add the end node
G.add_node("end")

* Now we need to add the edges. 

* There are a lot of them, so we'll want to use some for loops.

* The best way to use for loops depends on the shortest/longest path representation of the DP.

* For example, looking above, we can add all the red edges of length 0 &mdash; corresponding to not taking the item &mdash; in one fell swoop, like this:

In [ ]:
# Add edges corresponding to not taking an item
# (red edges of length 0)


In [7]:
# Solution
# Add edges corresponding to not taking an item
# (red edges of length 0)
for t in range(1, 4):
    for n in range(0, 9):
        G.add_edge((t, n), (t + 1, n), length=0)

* Next, we can add the green edges of length 11, corresponding to taking item 1 (gold). Don't forget our DP is a _longest_ path problem!

In [ ]:
# Add edges corresponding to taking item 1
# (green edges of length 11)


In [8]:
# Solution
# Add edges corresponding to taking item 1
# (green edges of length 11)
for n in range(3, 9):
    G.add_edge((1, n), (2, n - 3), length=-11)

* We can do something similar for the light blue and orange edges as well:

In [ ]:
# Add edges corresponding to taking item 2
# (light blue edges of length 7)

# Add edges corresponding to taking item 3
# (orange edges of length 12)


In [9]:
# Solution
# Add edges corresponding to taking item 2
# (light blue edges of length 7)
for n in range(2, 9):
    G.add_edge((2, n), (3, n - 2), length=-7)

# Add edges corresponding to taking item 3
# (orange edges of length 12)
for n in range(4, 9):
    G.add_edge((3, n), (4, n - 4), length=-12)

* Finally, we can add the edges from the last stage nodes to the special "end" node:

In [ ]:
# Add edges from stage 4 to the end node


In [10]:
# Solution
# Add edges from stage 4 to the end node
for n in range(0, 9):
    G.add_edge((4, n), "end", length=0)

* Now, we can solve the dynamic program using the Bellman-Ford algorithm, just as before:

In [ ]:
# Solve DP by solving its shortest path representation using Bellman-Ford


In [11]:
# Solution
# Solve DP by solving its shortest path representation using Bellman-Ford
length, nodes, negative_cycle = bf.bellman_ford(G, source=(1, 8), target="end", weight="length")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Shortest path length: -23
Shortest path: [(1, 8), (2, 5), (3, 5), (4, 1), 'end']


### Interpreting the output

* What is the maximum value we can carry in the knapsack?

_Write your notes here. Double-click to edit._

_Solution._ 
The maximum value we can carry in the knapsack is 23, the negative of the shortest path length.

* Which items should we take to obtain this maximum value?

_Write your notes here. Double-click to edit._

_Solution._ According to the edges in the shortest path, we should take the gold and platinum, but not the silver.

## Practice makes perfect &mdash; on your own

* Here are three more examples of dynamic programs we modeled in a previous lesson. Solve them using NetworkX and interpret the output.

### Assigning patrol cars to precincts

<!-- Winston and Venkataramanan Problem 13.4.4 -->
The Simplexville Police Department wants to determine how to assign patrol cars to each precinct in Simplexville. Each precinct can be assigned 0, 1, or 2 patrol cars. The number of crimes in each precinct depends on the number of patrol cars assigned to each precinct:
                                      
| Precinct | 0 patrol cars | 1 patrol cars | 2 patrol cars | 
| -------- | ------------- | ------------- | ------------- | 
| 1 | 14 | 10 | 7 |
| 2 | 25 | 19 | 17 |
| 3 | 20 | 14 | 11 |
                                      
The department has 5 patrol cars. The department's goal is to minimize the total number of crimes across all 3 precincts. 

* We formulated this problem as a dynamic program with the following shortest path representation:
    - Stage $t$ represents the decision to assign patrol cars to precinct $t$ $(t = 1, 2, 3)$ or the end of the decision-making process ($t = 4$).
    - Node $t_n$ represents having $n$ patrol cars left at stage $t$ ($n = 0, 1, \dots, 5$).

![DP for patrol car example](img/patrol.png)

Solve this dynamic program using NetworkX.

In [12]:
# Solution
# Create empty graph
G = nx.DiGraph()

# Add the stage-state nodes
for t in range(1, 5):
    for n in range(0, 6):
        G.add_node((t, n))

# Add the end node
G.add_node("end")

# Add edges corresponding to adding 0 patrol cars - red edges
for n in range(0, 6):
    # precinct 1: length 14
    G.add_edge((1, n), (2, n), length=14)
    
    # precinct 2: length 25
    G.add_edge((2, n), (3, n), length=25)
    
    # precinct 3: length 20
    G.add_edge((3, n), (4, n), length=20)

# Add edges corresponding to adding 1 patrol car - orange edges
for n in range(1, 6):
    # precinct 1: length 10
    G.add_edge((1, n), (2, n - 1), length=10)

    # precinct 2: length 19
    G.add_edge((2, n), (3, n - 1), length=19)
    
    # precinct 3: length 14
    G.add_edge((3, n), (4, n - 1), length=14)
    
# Add edges corresponding to adding 2 patrol cars - green edges
for n in range(2, 6):
    # precinct 1: length 7
    G.add_edge((1, n), (2, n - 2), length=7)
    
    # precinct 2: length 17    
    G.add_edge((2, n), (3, n - 2), length=17)    

    # precinct 3: length 11 
    G.add_edge((3, n), (4, n - 2), length=11)
    
# Add edges from last stage to the end node
for n in range(0, 6):
    G.add_edge((4, n), "end", length=0)
    
# Solve DP by solving its shortest path representation using Bellman-Ford
length, nodes, negative_cycle = bf.bellman_ford(G, source=(1, 5), target="end", weight="length")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Shortest path length: 37
Shortest path: [(1, 5), (2, 3), (3, 2), (4, 0), 'end']


_Interpret the output of the DP here. Double-click to edit._

_Solution &mdash; Interpretation._

* The minimum number of crimes as a result of assigning the 5 patrol cars to the 3 precincts is 37, the shortest path length.

* To achieve this minimum number of crimes, assign 2 patrol cars to precinct 1, 1 patrol car to precinct 2, and 2 patrol cars to precinct 3.

### Inventory management

<!-- Rardin Exercise 9-26 -->
The Dijkstra Brewing Company is planning production of its new limited run beer, Primal Pilsner. The company must supply 1 batch next month, then 2 and 4 in successive months. Each month in which the company produces the beer requires a factory setup cost of \\$5,000. Each batch of beer costs \\$2,000 to produce. Batches can be held in inventory at a cost of \\$1,000 per batch per month. Capacity limitations allow a maximum of 3 batches to be produced during each month. In addition, the size of the company's warehouse restricts the ending inventory for each month to at most 3 batches. The company has no initial inventory.
  
The company wants to find a production plan that will meet all demands on time and minimizes its total production and holding costs over the next 3 months. 

* We formulated this problem as a dynamic program with the following shortest path representation:
    - Stage $t$ represents deciding to produce in month $t$ ($t = 1, 2, 3$), or the end of the decision-making process ($t = 4$).
    - Node $t_n$ represents having $n$ batches in inventory at the end of stage $t$ ($n = 0, 1, 2, 3$).

![DP for inventory management example](img/inventory.png)

Solve this dynamic program using NetworkX.

In [13]:
# Solution
# Create empty graph
G = nx.DiGraph()

# Add the stage-state nodes
for t in range(1, 5):
    for n in range(0, 3):
        G.add_node((t, n))

# Add the end node
G.add_node("end")

# Add edges corresponding to production in month 1
# 0 batches: green edges
for n in range(1, 4):
    G.add_edge((1, n), (2, n - 1), length=1*(n - 1))

# 1 batch: blue edges
for n in range(0, 4):
    G.add_edge((1, n), (2, n), length=5 + 2*(1) + 1*(n))

# 2 batches: orange edges
for n in range(0, 3):
    G.add_edge((1, n), (2, n + 1), length=5 + 2*(2) + 1*(n + 1))

# 3 batches: purple edges
for n in range(0, 2):
    G.add_edge((1, n), (2, n + 2), length=5 + 2*(3) + 1*(n + 2))
    
# Add edges corresponding to production in month 2
# 0 batches: green edges
for n in range(2, 4):
    G.add_edge((2, n), (3, n - 2), length=1*(n - 2))

# 1 batch: blue edges
for n in range(1, 4):
    G.add_edge((2, n), (3, n - 1), length=5 + 2*(1) + 1*(n - 1))

# 2 batches: orange edges
for n in range(0, 4):
    G.add_edge((2, n), (3, n), length=5 + 2*(2) + 1*(n))

# 3 batches: purple edges
for n in range(0, 3):
    G.add_edge((2, n), (3, n + 1), length=5 + 2*(3) + 1*(n + 1))
    
# Add edges corresponding to production in month 3
# 0 batches: not possible!

# 1 batch: blue edges
for n in range(3, 4):
    G.add_edge((3, n), (4, n - 3), length=5 + 2*(1) + 1*(n - 3))

# 2 batches: orange edges
for n in range(2, 4):
    G.add_edge((3, n), (4, n - 2), length=5 + 2*(2) + 1*(n - 2))

# 3 batches: purple edges
for n in range(1, 4):
    G.add_edge((3, n), (4, n - 1), length=5 + 2*(3) + 1*(n - 1))
  
# Add edges from last stage to the end node
for n in range(0, 4):
    G.add_edge((4, n), "end", length=0)
    
# Solve DP by solving its shortest path representation using Bellman-Ford
length, nodes, negative_cycle = bf.bellman_ford(G, source=(1, 0), target="end", weight="length")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

Shortest path length: 30
Shortest path: [(1, 0), (2, 0), (3, 1), (4, 0), 'end']


_Interpret the output of the DP here. Double-click to edit._

_Solution &mdash; Interpretation._

* The minimum total production and holding cost over the next 3 months is 30.

* To achieve this minimum cost, the company should produce 1 batch in month 1, 3 batches in month 2, and 3 batches in month 3.

### Study time

To graduate from Simplexville University, Angie needs to pass at least one of
the three courses she is taking this semester: literature, finance, and
statistics. Angie's busy schedule of extracurricular activities allows her to
spend only 4 hours per week on studying. Angie's probability of passing each
course depends on the number of hours she spends studying for the course:

| Hours of studying per week | Literature | Finance | Statistics |
|:--------------------------:|------------|---------|------------|
| 0                          | 0.20       | 0.25    | 0.10       |
| 1                          | 0.30       | 0.30    | 0.30       |
| 2                          | 0.35       | 0.33    | 0.40       |
| 3                          | 0.38       | 0.35    | 0.44       |
| 4                          | 0.40       | 0.38    | 0.50       |

Angie wants to maximize the probability that she passes at least one of these
three courses. Formulate this problem as a dynamic program by giving its
shortest/longest path representation.

- We formulated this problem as a dynamic program with the following shortest path representation:
    - Stage $t$ represents assigned time to course $t$ ($t = 1, 2, 3$) or the end of the decision-making process ($t = 4$).
    - Node $t_n$ represents having $n$ hours left to assign at stage $t$ ($n = 0, 1, 2, 3, 4$).




_Hint._ You can import the natural exponent and logarithm functions from the `math` library:

```python
from math import exp, log
```

![DP for study time example](img/study.png)

Solve this dynamic program using NetworkX.

In [14]:
# Solution
# Import natural exponent and logarithm functions from math library
from math import exp, log

# Create empty graph
G = nx.DiGraph()

# Add the stage-state nodes
for t in range(1, 5):
    for n in range(0, 5):
        G.add_node((t, n))

# Add the end node
G.add_node("end")

# Create a dictionary of edge lengths
# Each key is a tuple: (course, hours)
edge_lengths = {
    (1, 0): log(0.80),
    (1, 1): log(0.70),
    (1, 2): log(0.65),
    (1, 3): log(0.62),
    (1, 4): log(0.60),
    (2, 0): log(0.75),
    (2, 1): log(0.70),
    (2, 2): log(0.67),
    (2, 3): log(0.65),
    (2, 4): log(0.62),
    (3, 0): log(0.90),
    (3, 1): log(0.70),
    (3, 2): log(0.60),
    (3, 3): log(0.56),
    (3, 4): log(0.50)
}

# Add edges from each stage t and state n
for t in range(1, 4):
    for n in range(0, 5):
        for m in range(0, 5):  # m = number of hours to assign
            if m <= n:  # Check if number of hours to assign <= number of hours left
                G.add_edge((t, n), (t + 1, n - m), length=edge_lengths[t, m])

# Add edges from last stage to end node
for n in range(0, 5):
    G.add_edge((4, n), "end", length=0)

# Solve DP by solving its shortest path representation using Bellman-Ford
length, nodes, negative_cycle = bf.bellman_ford(G, source=(1, 4), target="end", weight="length")
print(f"Shortest path length: {length}")
print(f"Shortest path: {nodes}")

# Convert shortest path length using inverse log (exp)
print(f"Minimum probability of failing all 3 courses: {exp(length)}")

Shortest path length: -1.2292906123102258
Shortest path: [(1, 4), (2, 2), (3, 2), (4, 0), 'end']
Minimum probability of failing all 3 courses: 0.2925


_Interpret the output of the DP here. Double-click to edit._

_Solution &mdash; Interpretation._

* The minimum probability of failing all 3 courses is $0.2925$. Equivalently, the maximum probability of passing at least 1 course is $1 - 0.2925 = 0.7075$.

* To achieve this minimum probability of failing all 3 courses, Angie should assign 2 hours/week to Literature (Course 1) and 2 hours/week to Statistics (Course 3).

---

## Problems

### Problem 1 (Dynamic Distillery, revisited)

You have been put in charge of launching Dynamic Distillery's new bourbon whiskey. There are 4 nonoverlapping phases: research, development, manufacturing system design, and initial production and distribution. Each phase can conducted the two speeds: normal or priority. The times required (in months) to complete each phases at the two speeds are:

| Level    | Research | Development | Manufacturing System Design | Initial Production and Distribution |
|:---------|:---------|:------------|:----------------------------|:------------------------------------|
| Normal   | 4        | 3           | 5                           | 2                                   |
| Priority | 2        | 2           | 3                           | 1                                   |

The costs (in millions of \$) of complete each phase at the two speeds are:

| Level    | Research | Development | Manufacturing System Design | Initial Production and Distribution |
|:---------|:---------|:------------|:----------------------------|:------------------------------------|
| Normal   | 2        | 2           | 3                           | 1                                   |
| Priority | 3        | 3           | 4                           | 2                                   |

You have been given \$10 million dollars to execute the launch as quickly as possible. 

Once upon a time, for homework, you formulated this problem as a dynamic program by giving its shortest/longest path representation.

1. Solve your dynamic program using NetworkX.
2. Interpret the output of your dynamic program.

In [15]:
# Solution
# Create empty graph
G = nx.DiGraph()

# Add stage-state nodes
for t in range(1, 6):
    for n in range(0, 11):
        G.add_node((t, n))

# Add "end" node
G.add_node("end")

# Stage 1 at normal speed
for n in range(2, 11):
    G.add_edge((1, n), (2, n - 2), length=4)

# Stage 1 at priority speed
for n in range(3, 11):
    G.add_edge((1, n), (2, n - 3), length=2)

# Stage 2 at normal speed
for n in range(2, 11):
    G.add_edge((2, n), (3, n - 2), length=3)

# Stage 2 at priority speed
for n in range(3, 11):
    G.add_edge((2, n), (3, n - 3), length=2)

# Stage 3 at normal speed
for n in range(3, 11):
    G.add_edge((3, n), (4, n - 3), length=5)

# Stage 3 at priority speed
for n in range(4, 11):
    G.add_edge((3, n), (4, n - 4), length=3)

# Stage 4 at normal speed
for n in range(1, 11):
    G.add_edge((4, n), (5, n - 1), length=2)

# Stage 4 at priority speed
for n in range(2, 10):
    G.add_edge((4, n), (5, n - 2), length=1)

# Add edges from last stage to end node
for n in range(0, 11):
    G.add_edge((5, n), "end", length=0)

# Solve shortest path problem
length, nodes, negative_cycle = bf.bellman_ford(G, source=(1, 10), target="end", weight="length")
print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length = {length}")
print(f"Shortest path = {nodes}")

Negative cycle? False
Shortest path length = 10
Shortest path = [(1, 10), (2, 7), (3, 5), (4, 1), (5, 0), 'end']


_Interpret the output of the DP here. Double-click to edit._

_Solution &mdash; Interpretation._

* The earliest the launch can happen is in 10 weeks, which is the shortest path length in this case.

* In order to launch in 10 weeks, research and manufacturing system design should be done at the priority speed, while development and inital production and distribution should be done at normal speed.

### Problem 2 (Pear Computers, revisited)

Pear Computers has a contract to deliver the following number of laptop computers during the next three months:

|                           | Month 1 | Month 2 | Month 3 |
|:--------------------------|:--------|:--------|:--------|
| Laptop computers required | 200     | 300     | 200     |

For each laptop produced during months 1 and 2, a \\$100 cost is incurred; for each laptop produced during month 3, a \\$120 cost is incurred. Each month in which the company produces laptops requires a factory setup cost of \\$2,500. Laptops can be held in a warehouse at a cost of \\$15 for each laptop in inventory at the end of a month. The warehouse can hold at most 400 laptops. 

Laptops made during a month may be used to meet demand for that month or any future month. Manufacturing constraints require that laptops be produced in multiples of 100, and at most 300 laptops can be produced in any month.  The company's goal is to find a production plan that will meet all demands on time and minimizes its total production and holding costs over the next 3 months.

Formulate this problem as a dynamic program by giving its shortest/longest path representation.

Once upon a time, for homework, you formulated this problem as a dynamic program by giving its shortest/longest path representation.

1. Solve your dynamic program using NetworkX.
2. Interpret the output of your dynamic program.

In [16]:
# Solution
# Create empty graph
G = nx.DiGraph()

# Add stage-state nodes
for t in range(1, 5):
    for n in range(0, 5):
        G.add_node((t, n))

# Add "end" node
G.add_node("end")

# Month 1
# Production amount = 0
for n in range(2, 5):
    G.add_edge((1, n), (2, n - 2), length=15*100*(n - 2))

# Production amount = 100
for n in range(1, 5):
    G.add_edge((1, n), (2, n - 1), length=2500 + 100*100 + 15*100*(n - 1))

# Production amount = 200
for n in range(0, 5):
    G.add_edge((1, n), (2, n), length=2500 + 100*200 + 15*100*n)

# Production amount = 300
for n in range(0, 4):
    G.add_edge((1, n), (2, n + 1), length=2500 + 100*300 + 15*100*(n + 1))

# Month 2
# Production amount = 0
for n in range(3, 5):
    G.add_edge((2, n), (3, n - 3), length=15*100*(n - 3))

# Production amount = 100
for n in range(2, 5):
    G.add_edge((2, n), (3, n - 2), length=2500 + 100*100 + 15*100*(n - 2))

# Production amount = 200
for n in range(1, 5):
    G.add_edge((2, n), (3, n - 1), length=2500 + 100*200 + 15*100*(n - 1))

# Production amount = 300
for n in range(0, 5):
    G.add_edge((2, n), (3, n), length=2500 + 100*300 + 15*100*n)

# Month 3
# Production amount = 0
for n in range(2, 5):
    G.add_edge((3, n), (4, n - 2), length=15*100*(n - 2))

# Production amount = 100
for n in range(1, 5):
    G.add_edge((3, n), (4, n - 1), length=2500 + 120*100 + 15*100*(n - 1))

# Production amount = 200
for n in range(0, 5):
    G.add_edge((3, n), (4, n), length=2500 + 120*200 + 15*100*n)

# Production amount = 300
for n in range(0, 4):
    G.add_edge((3, n), (4, n + 1), length=2500 + 120*300 + 15*100*(n + 1))

# Add edges from last stage to end node
for n in range(0, 5):
    G.add_edge((4, n), "end", length=0)

# Solve shortest path problem
length, nodes, negative_cycle = bf.bellman_ford(G, source=(1,0), target="end", weight="length")
print(f"Negative cycle? {negative_cycle}")
print(f"Shortest path length = {length}")
print(f"Shortest path = {nodes}")

Negative cycle? False
Shortest path length = 81500
Shortest path = [(1, 0), (2, 0), (3, 0), (4, 0), 'end']


_Interpret the output of the DP here. Double-click to edit._

_Solution &mdash; Interpretation._

* The minimum production and holding cost required to meet demand over the next 3 months is 81,500, which is the shortest path length in this case.

* In order to meet this minimum total cost, the company should produce 200 in month 1, produce 300 in month 2, and produce 200 in month 3.